In [30]:
import skyfield
from skyfield.api import EarthSatellite, load, wgs84, N, S, E, W
import numpy as np
from math import degrees

from astropy import units as u

from poliastro.bodies import Earth, Mars, Sun
from poliastro.twobody import Orbit

# from skyfield.timelib import Time
# from mpl_toolkits.mplot3d import Axes3D

#### 3D plotter

from poliastro.examples import churi, iss, molniya
from poliastro.plotting import OrbitPlotter3D

import sys
path = '/home/virgil/Documents/Obsidian/Python/0-My_module'
path2 = '/home/virgil/Documents/Obsidian/Python/3-CSUM'
sys.path.append(path)
sys.path.append(path2)
import fct_file
import fct_csum as fctcsum

In [31]:
# Useful for defining quantities
from astropy import units as u

# Earth focused modules, ISS example orbit and time span generator
import poliastro.earth
from poliastro.earth.plotting import GroundtrackPlotter
from poliastro.examples import iss
from astropy.time import Time
from poliastro.util import time_range
import plotly.graph_objects as go

In [32]:
def compare_sats(sat1, sat2):
    for attr in sorted(dir(sat1.model)):
        try:
            value1 = getattr(sat1.model, attr)
        except AttributeError:
            continue
        if not isinstance(value1, (str, int, float)):
            continue
        value2 = getattr(sat2.model, attr)
        verdict = '==' if (value1 == value2) else '!='
        print('{:18} {!r:24} {} {!r:24}'.format(attr, value1, verdict, value2))

In [33]:
def sat_to_orb(sat):
    mu = 3.9860044188e14 #[m3/s2]
    Rt = 6371 #km

    mean_mo = sat.model.no_kozai
    P_orbit = 2 * np.pi / mean_mo * 60 #min

    h = (mu / (4 * np.pi**2) * P_orbit ** 2 )**(1/3) / 1000 - Rt   ## Altitude of an orbit
    Ma = sat.model.mo
    e = sat.model.ecco
    tru_anomaly = Ma + (2*e - 1/4 *e**3) * np.sin(Ma) + 5/4 * e**2 * np.sin(2*Ma) + 13/12 * e**3 * np.sin(3*Ma)
    r_earth = Earth.R_mean << u.km
    h_orbit = h << u.km 
    a =  r_earth + h_orbit
    ecc = e << u.one  ### not used ?
    inc = degrees(sat.model.inclo) << u.deg
    raan = degrees(sat.model.nodeo) << u.deg
    argp = degrees(sat.model.argpo) << u.deg
    nu = degrees(tru_anomaly) << u.deg
    orb = Orbit.from_classical(Earth, a, ecc, inc, raan, argp, nu, sat.epoch.to_astropy())
    return(orb)

In [34]:
# Time(sat1.epoch.tai, format='tai')

## Orbit

In [35]:
TLE1 = """
1 99999U 23099A 24273.64765046  .00000000  00000-0  00000-0 0  0000
2 99999 061.9897 239.0740 0018052 327.0019 000.0000 630.9293807100000
"""

TLE2 = """
1 60243U 24128J   24272.91320405  .00017738  00000+0  15177-2 0  9991
2 60243  61.9898 241.6152 0018018 326.5644  33.4305 14.97602210 12134
"""


In [36]:
sat1 = fctcsum.TLE_to_sat(TLE1[1:-1])
sat2 = fctcsum.TLE_to_sat(TLE2[1:-1])

In [37]:
orb_sat1 = sat_to_orb(sat1)
orb_sat2 = sat_to_orb(sat2)

In [38]:
compare_sats(sat1, sat2)

Om                 4.172628455912924        != 4.216980762864603       
__doc__            'High-speed computation of satellite positions and velocities.' == 'High-speed computation of satellite positions and velocities.'
__module__         'sgp4.wrapper'           == 'sgp4.wrapper'          
a                  0.08802553308366504      != 1.0898757985977732      
alta               -0.9118155632240124      != 0.09183953681168666     
altp               -0.9121333706086576      != 0.08791206038385968     
am                 0.08802553308366504      != 1.0898757985977732      
argpdot            -0.06065914283729058     != 4.501778483249792e-06   
argpo              5.707259815277245        != 5.69962399979977        
bstar              0.0                      != 0.0015176999999999999   
classification     'U'                      == 'U'                     
ecco               0.0018052                != 0.0018018               
elnum              0                        != 999        

In [20]:
#### 3D plotter

from poliastro.examples import churi, iss, molniya
from poliastro.plotting import OrbitPlotter3D
from poliastro.twobody import Orbit

frame = OrbitPlotter3D()

frame.plot(orb_sat1)
frame.plot(orb_sat2)

In [21]:
poliastro_sat1 = poliastro.earth.EarthSatellite(orb_sat1,None)
poliastro_sat2 = poliastro.earth.EarthSatellite(orb_sat2,None)

In [22]:
t_span = time_range(poliastro_sat1.orbit.epoch - (0.5 * u.h), periods=150, end=poliastro_sat1.orbit.epoch + (2.5 * u.h))

In [23]:
STATION = [43.637, 3.843] * u.deg 

In [24]:
# Generate an instance of the plotter, add title and show latlon grid
gp = GroundtrackPlotter()
gp.update_layout(title="Comparison")

# Plot previously defined EarthSatellite object
gp.plot(
    poliastro_sat1,
    t_span,
    label="old_TLE",
    color="red",
    marker={
    "size": 10,
    "symbol": "triangle-right",
    "line": {"width": 1, "color": "black"},
},
)
gp.plot(
    poliastro_sat2,
    t_span,
    label="new_TLE",
    color="blue",
    # line_style = ':',
    marker={
    "size": 10,
    "symbol": "triangle-right",
    "line": {"width": 2, "color": "black"},
},

)

gp.add_trace(
    go.Scattergeo(
        lat=STATION[0],
        lon=STATION[-1],
        name="CSUM",
        marker={"color": "blue"},
    )
)
gp.fig.show()b

In [25]:
gp.update_geos(projection_type="orthographic")
gp.fig.show()

In [37]:
from astropy import units as u
from poliastro.util import time_range

# Function to get lat, lon, alt from a satellite
def get_lat_lon_alt(satellite, time):
    r = satellite.attractor.ellipsoid
    lat, lon, alt = r.subpoint(satellite.state, time)
    return lat.deg, lon.deg, alt

# Generate an instance of the plotter, add title, and show latlon grid
gp = GroundtrackPlotter()
gp.update_layout(title="Comparison", showlegend=True)

# Plot previously defined EarthSatellite objects
gp.plot(
    poliastro_sat1,
    t_span,
    label="old_TLE",
    color="red",
    marker={
        "size": 10,
        "symbol": "triangle-right",
        "line": {"width": 1, "color": "black"},
    },
)

gp.plot(
    poliastro_sat2,
    t_span,
    label="new_TLE",
    color="blue",
    marker={
        "size": 10,
        "symbol": "triangle-right",
        "line": {"width": 2, "color": "black"},
    },
)

# Add annotations for satellite positions and dates at specific intervals
for time in t_span:
    # Get lat/lon for both satellites at the current time step
    lat1, lon1, alt1 = get_lat_lon_alt(poliastro_sat1, time)
    lat2, lon2, alt2 = get_lat_lon_alt(poliastro_sat2, time)

    # Add text annotation for each satellite position and time
    gp.add_trace(
        go.Scattergeo(
            lat=[lat1],
            lon=[lon1],
            text=[f"old_TLE<br>Lat: {lat1:.2f}°, Lon: {lon1:.2f}°<br>{time.utc.iso}"],
            mode="markers+text",
            marker=dict(size=8, color="red"),
            textposition="top right",
            name=f"old_TLE - {time.utc.iso}",
        )
    )
    gp.add_trace(
        go.Scattergeo(
            lat=[lat2],
            lon=[lon2],
            text=[f"new_TLE<br>Lat: {lat2:.2f}°, Lon: {lon2:.2f}°<br>{time.utc.iso}"],
            mode="markers+text",
            marker=dict(size=8, color="blue"),
            textposition="top left",
            name=f"new_TLE - {time.utc.iso}",
        )
    )

# Adding station position on map
gp.add_trace(
    go.Scattergeo(
        lat=[STATION[0]],
        lon=[STATION[1]],
        name="Station",
        marker={"color": "green", "size": 12, "symbol": "star"},
    )
)

# Show the plot
gp.fig.show()


AttributeError: 'EarthSatellite' object has no attribute 'attractor'